In [2]:
def clearall():
    all = [var for var in globals() if var[0] != "_"]
    for var in all:
        del globals()[var]
        
clearall()

In [3]:
############################
######## LIBRAIRIES ########
############################
print('Importing Librairies...')
from sklearn.linear_model import LinearRegression
import colorama as col
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import time
import os
import datetime as dt
import numpy as np
import fxcmpy
import socketio
import shelve
import talib
import statistics
import pyttsx3

engine = pyttsx3.init()

print('Librairies imported\n')
print('version fxcmpy :',fxcmpy.__version__)
%matplotlib inline
plt.style.use('seaborn')
engine.say("librairie loaded")
engine.runAndWait()
print('Prêt')

Importing Librairies...
Librairies imported

version fxcmpy : 1.2.6
Prêt


In [4]:
engine.say("Ignition du Lumberjack")
engine.runAndWait() 

##############################################
######## INITIALISATION DES VARIABLES ########
##############################################

'''
# COMPTE TEST #

TOKEN = '89410007226e1ad8ffce5dbf03f1e1bf056f2451'
server = 'demo'
user_id = 'D261219574'
compte = '01215057'
password = '3555'
'''


# COMPTE DEV #
TOKEN = '4973fff4a559fa6e5a25dfd3e7c8f9367f04378b'
server = 'demo'
user_id = 'D161435731'
compte = '01425123'
password = '8889'


'''
# COMPTE POUBELLE #
TOKEN = 'a66bf5c0ec732487f4b348baf6364d3ee8784935'
server = 'demo'
user_id = 'D261219576'
compte = '01215059'
password = '8029'
'''


# minutes: 'm1' , 'm5' , 'm15' , 'm30' 
# hours: 'H1' , 'H2', 'H3', 'H4', 'H6' 'H8'
# one day: 'D1'
# one week: 'W1'
# one month: 'M1'


_number = 10000
_period = 'm30'
_period2 = 'H1'
_period3 = 'D1'
_total = 0
_cash_ini = 200000
_cash = _cash_ini
_rate = 1
_flag = 0


_size = 100
_target = 15
_sl = -30
_rsi_high = 80
_rsi_low = 20



engine.say("בסדר")
engine.runAndWait()
print('Variables initialisées')

Variables initialisées


In [5]:

############################
######## FONCTIONS ########
###########################


def conX():
    con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error',server=server)
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
    return(con)

def deconX():
    con = con.close()
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion non intérrompue'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion intérrompue'+col.Style.RESET_ALL)
    return()


def scrap_hist(_ticker,number,laperiod):
    print(col.Fore.BLUE,'Scraping des données OHLC pour le ticker',col.Fore.YELLOW,_ticker,col.Style.RESET_ALL)
    data = con.get_candles(instrument=_ticker,number = 10000,period=laperiod)
    df = pd.DataFrame(index=data.index)
    df['Open_Bid'] = data['bidopen']
    df['Open_Ask'] = data['askopen']
    df['Open'] = (data['bidopen']+data['askopen'])/2
    df['High_Bid'] = data['bidhigh']
    df['High_Ask'] = data['askhigh']
    df['High'] = (data['bidhigh']+data['askhigh'])/2
    df['Low_Bid'] = data['bidlow']
    df['Low_Ask'] = data['asklow']
    df['Low'] = (data['bidlow']+data['asklow'])/2
    df['Close_Bid'] = data['bidclose']
    df['Close_Ask'] = data['askclose']
    df['Close'] = (data['bidclose']+data['askclose'])/2
    return(df)

def convert(devise,_rate,_flag):
    
    prix_usd = 0
    if x[4:] == 'USD':
        prix_usd = devise
    elif x[:3] == 'USD':
        prix_usd = round(devise/_rate,2)
    elif _flag == 1:
        prix_usd = round(devise * _rate,2)
       
    elif _flag == -1 :
            prix_usd = round(devise / _rate,2)
    else:
        prix_usd = 'NA'
    return(prix_usd,_flag)

def peak_detect(price,order=10):
    max_idx = list(argrelextrema(price,np.greater,order=order)[0])
    min_idx = list(argrelextrema(price,np.less,order=order)[0])
    idx = max_idx + min_idx + [len(price)-1]
    idx.sort()
    _current_idx = idx[-3:]
    
    _start = min(_current_idx)
    _end = max(_current_idx)
    _current_pat = price[_current_idx]
    return _current_idx,_current_pat,_start,_end 
    
def is_BAAM(moves,err_allowed):  
    XA=moves[0]
    AB=moves[1]
    
    
    X=prices[0]
    A=prices[1]
    B=prices[2]
    
    AB_range = np.array([0.382 - err_allowed,0.382+ err_allowed])*abs(XA)
    AB_range2 = np.array([0.5 - err_allowed,0.5+ err_allowed])*abs(XA)
    
    
    
    if abs(A-X)/X>0.002:
        
        if XA>0 and AB<0 :

            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]) :

                BAAM = 1
                return (BAAM)
            else:
                return (np.isnan)

        elif XA<0 and AB>0 :


            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]):

                BAAM = -1
                return (BAAM)
            else:
                return (np.isnan)
        else:
             return (np.isnan)       
            
 
  

engine.say("functions initiated")
engine.runAndWait()
print('Functions initiated')

Functions initiated


In [6]:
#########################
### CONNEXION A L'API ###
#########################

___Author___='LumberJack Jyss'
print('Global Optimized LumberJack Environment Motor for FOR_EX\nLumberJack Jyss 5780(c)')
print(col.Fore.BLUE,'°0Oo_D.A.G._26_oO0°')
print(col.Fore.YELLOW,col.Back.BLUE,'--- Go!em MOVING AVERAGE RETRACEMENT #v0.15 ---',col.Style.RESET_ALL)

print('')
engine.say(" Initialisation du Gaulem Moving, Average, Retracement")
engine.say("Connexion du Gaulem hà la Péh e")
engine.runAndWait()

try:
    con.is_connected() == True
    
    engine.say("already Connected")
    engine.runAndWait()
    print(col.Fore.GREEN+'Connexion rétablie'+col.Style.RESET_ALL)
    print('Compte utilisé : ',con.get_account_ids())
    print('')
    
except:
    try:
        con = conX()
        con.is_connected() == True
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
        engine.say("Connected")
        engine.runAndWait()
    except:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
        engine.say("Not Connected, sal rass de mor, pitun cé la merd")
        engine.say("vérifi ton internet, é relance le gaulèm")
        engine.runAndWait()
        print('')
        #os._exit(0)
        con = deconX()
        time.sleep(1)
        con = conX()

Global Optimized LumberJack Environment Motor for FOR_EX
LumberJack Jyss 5780(c)
 °0Oo_D.A.G._26_oO0°
  --- Go!em MOVING AVERAGE RETRACEMENT #v0.15 --- 

Connexion établie
Compte utilisé :  [1425123]
Connexion établie
Compte utilisé :  [1425123]


In [8]:
DATE = []
CONTRACT = []
OPEN_POZ = []
CLOSE_POZ = []
RATE_OPEN_POZ = []
RATE_CLOSE_POZ = []
PNL_LAT = []
PNL_REAL = []
TOTAL_OPEN = []
TOTAL_CLOSE = []
PRICE_BUY = []
PRICE_SELL = []
DER_POZ = []
TOTAL_PNL_LAT = []
TOTAL_PNL_REAL = []
_cash = _cash_ini

In [9]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import pandas as pd
from numpy import loadtxt
from functools import reduce
import talib
import time
from scipy.signal import argrelextrema
from ipywidgets import *

from datetime import datetime

In [10]:
TICKERS = con.get_instruments()

In [11]:
%%time
df_resultats = pd.DataFrame(index=['TimeFrame 1','TimeFrame2','Equity','Nbre Winners','Nbre Loosers','Average lenght of trade','Cumul pnl'])

#TICKERS = con.get_instruments()
TICKERS = [ 'EUR/USD','GBP/USD','EUR/GBP','EUR/JPY','GBP/JPY','EUR/AUD','EUR/CAD','AUD/JPY','CAD/JPY','NZD/JPY','GBP/CAD','GBP/AUD','AUD/NZD','EUR/SEK','EUR/NZD']

engine.say("קדימה")
engine.runAndWait()
j = 0
k = 0
err_allowed = 2/100
pips = np.array([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])
plt.ion()
position = 0
equity = 100000
for x in TICKERS:

    if x[4:] == 'USD':
        _rate = 1

    elif x[:3] == 'USD':
        con.subscribe_market_data(x)
        _rate = con.get_last_price(x).Bid
        con.unsubscribe_market_data(x)
    elif x[:3] != 'USD' and x[4:] != 'USD':
        try:
            con.subscribe_market_data(x[4:]+'/USD')
            _rate = con.get_last_price(x[4:]+'/USD').Bid
            con.unsubscribe_market_data(x[4:]+'/USD')
            _flag = 1
        except:
            con.subscribe_market_data('USD/'+x[4:])
            _rate = con.get_last_price('USD/'+x[4:]).Bid
            con.unsubscribe_market_data('USD/'+x[4:])
            _flag = -1
    else:
        prix_usd = 'NA'

    print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)
    label = ''
    data = scrap_hist(x,_number,_period)
    data2 = scrap_hist(x,_number,_period2)
    data3 = scrap_hist(x,_number,_period3)
    price = data.copy()
    price2 = data2.copy()
    price3 = data3.copy()
    _position = 0
    _equity = 0
    _nbtransactions = 0
    backtest_graph = pd.DataFrame()
    EQUITY = [_cash]
    CASH = [_cash]
    _winner = 0
    _looser = 0
    _index_entry = 0
    TRADE_DURATION = []
    _average_duration = 0
    _size = 50000

    PRICE_BUY = []
    PRICE_SELL = []
   
    _total = 0

    _target = 0.003
    _sl = 0.0015
    _spread_filter = 0.00050
    _filtre_multipoz = 0.1 # En %
    _filtre_multipoz = _filtre_multipoz/100
    _print_chart = 'no'
    _print_patterns = 'no' 

    _open_buy = 0
    _open_sell = 0
    _minipoz = 0
    
    MM21 = talib.EMA(price.Close, timeperiod=21)
    MM34 = talib.EMA(price.Close, timeperiod=34)
    MM54 = talib.EMA(price.Close, timeperiod=54)
    MM200 = talib.EMA(price.Close, timeperiod=200)
    rsi = talib.RSI(price.Close, timeperiod=14)
    rsi_high = 70
    rsi_low = 30
    upperband, middleband, lowerband = talib.BBANDS(price.Close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    fastk, fastd = talib.STOCHRSI(price.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)#STOCHASTICRSI
    slowk, slowd = talib.STOCH(price.High, price.Low, price.Close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)#STOCHASTIC
    
    MM82 = talib.EMA(price2.Close, timeperiod=8)
    MM212 = talib.EMA(price2.Close, timeperiod=21)
    MM342 = talib.EMA(price2.Close, timeperiod=34)
    MM542 = talib.EMA(price2.Close, timeperiod=54)
    rsi2 = talib.RSI(price2.Close, timeperiod=14)
    upperband2, middleband2, lowerband2 = talib.BBANDS(price2.Close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    fastk2, fastd2 = talib.STOCHRSI(price2.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)#STOCHASTICRSI
    slowk2, slowd2 = talib.STOCH(price2.High, price2.Low, price2.Close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

    MM83 = talib.EMA(price3.Close, timeperiod=8)
    MM213 = talib.EMA(price3.Close, timeperiod=21)
    MM343 = talib.EMA(price3.Close, timeperiod=34)
    rsi3 = talib.RSI(price3.Close, timeperiod=14)
    upperband3, middleband3, lowerband3 = talib.BBANDS(price3.Close, timeperiod=5, nbdevup=2, nbdevdn=2, matype=0)
    fastk3, fastd3 = talib.STOCHRSI(price3.Close, timeperiod=14, fastk_period=5, fastd_period=3, fastd_matype=0)#STOCHASTICRSI
    slowk3, slowd3 = talib.STOCH(price3.High, price3.Low, price3.Close, fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
    
    for i in range(150,len(price)-1):
        
        j = len(price2[price2.index <= price.index[i]])-1
        k = len(price3[price3.index <= price.index[i]])-1   
        
        current_idx,current_pat,start,end = peak_detect(price.Close.values[:i+1])
        #current_idx,current_pat,start,end = peak_detect(pr.High.values[:i+1],price.Low.values[:i+1],price.Close.values[:i+1])
        current_rsi = [rsi.iloc[current_idx[0]],rsi.iloc[current_idx[1]],rsi.iloc[current_idx[2]]]
        current_rsi_max = max(current_rsi)
        current_rsi_min = min(current_rsi)
        delta_price = current_pat[2] - current_pat[0]
        delta_rsi = current_rsi[2] - current_rsi[0]
      
        XA = current_pat[1] - current_pat[0]     
        AB = current_pat[2] - current_pat[1]      
       
        
        X = current_pat[0]
        A = current_pat[1]
        B = current_pat[2]
        

        prices =[X,A,B]

        

        moves=[XA,AB]
        
        BAAM = is_BAAM(moves,err_allowed)
       
       

        harmonics = np.array([BAAM])
       # harmonics = np.array([abcd])
        labels = ['BAAM']
    
        _minipoz = 0

        if _position == 0 :
            if (np.any(harmonics==1) and np.all(harmonics!=-1 )) or  (np.any(harmonics==-1) and np.all(harmonics!=1 )):

                for h in range(0, len(harmonics)):

                    if harmonics[h]==1:
                        sense = 'bullish' 
                    elif harmonics[h]==-1:
                        sense ='bearish' 
                        label =  sense + labels[h] +'found'

                    if _print_patterns == 'yes':


                        print(label)
                        print(i)
                    if _print_chart == 'yes':
                        plt.title(label)
                        plt.suptitle(i-current_idx[-1])
                        plt.figure(figsize=(16,6))
                        plt.plot(np.arange(start,i+15),price.Close.values[start:i+15])
                        plt.scatter(x=i,y=price.Close.values[i],c = "red")
                        plt.plot(current_idx,current_pat)
                        plt.show


                    if harmonics[h]==-1 and _minipoz == 0 and price.iloc[i]['Close']<MM21[i] and X>MM54[current_idx[0]] and MM21[current_idx[0]]>MM54[current_idx[0]] and MM21[i]<MM34[i]<MM54[i]\
                            and price2.Close[j]<MM542[j]:
                            _minipoz = -1
                            _pnl = 0
                            _open_sell += 1
                            _equity = _cash + _pnl
                            EQUITY.append(_equity)
                            CASH.append(_cash)
                            _index_entry = i
                            _position = -1
                            _nbtransactions += 1
                            price_sell = price.Close_Bid.iloc[i]
                            PRICE_SELL.append(price_sell)
                            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
                            print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')
                            print('Nouvelle moyenne du price_sell',_price_sell_mean)

                            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                            CONTRACT.append(x)
                            OPEN_POZ.append(-1)
                            CLOSE_POZ.append(0)
                            RATE_OPEN_POZ.append(price_sell)
                            RATE_CLOSE_POZ.append(0) 
                            PNL_LAT.append(_pnl)
                            PNL_REAL.append(0)
                            TOTAL_PNL_LAT.append(_pnl)
                            TOTAL_PNL_REAL.append(0)

                            TOTAL_OPEN.append(1)
                    
                    if harmonics[h]==1 and _minipoz == 0 and price.iloc[i]['Close']>MM21[i] and X<MM54[current_idx[0]] and MM21[current_idx[0]]<MM54[current_idx[0]] and MM21[i]>MM34[i]>MM54[i]\
                        and price2.Close[j]>MM542[j]:
                   
                        #and price.Close[i]>price.High[i-2] and price.Open[i]<price.Open[i-1]:
                            _minipoz = 1
                            _pnl = 0
                            _open_buy += 1
                            _equity = _cash + _pnl
                            EQUITY.append(_equity)
                            CASH.append(_cash)
                            _position = 1
                            _index_entry = i
                            _nbtransactions += 1
                            price_buy = price.Close_Ask.iloc[i]
                            PRICE_BUY.append(price_buy)
                            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
                            print('Position 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')
                            print('Nouvelle moyenne du price_buy',_price_buy_mean)

                            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                            CONTRACT.append(x)
                            OPEN_POZ.append(1)
                            CLOSE_POZ.append(0)
                            RATE_OPEN_POZ.append(price_buy)
                            RATE_CLOSE_POZ.append(0)
                            PNL_LAT.append(_pnl)
                            PNL_REAL.append(0)
                            TOTAL_OPEN.append(1) 
                            TOTAL_PNL_LAT.append(_pnl)
                            TOTAL_PNL_REAL.append(0) 

                    else :
                        _pnl = 0
                        _equity = _cash + _pnl
                        EQUITY.append(_equity)
                        CASH.append(_cash)
                
        elif _position == 1:

            
            if (price_buy - price.Close[i])/price_buy>=_filtre_multipoz and MM21[i]>MM34[i]>MM54[i]\
            and price2.Close[j]>MM542[j] :
             
                print(i)
                _minipoz = 1
                _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                _open_buy += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _position = 1
                _index_entry = i
                _nbtransactions += 1
                price_buy = price.Close_Ask.iloc[i]
                PRICE_BUY.append(price_buy)
                _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
                print('Position',_open_buy,'bought at ', price_buy,'(verification liste',PRICE_BUY[-1],')')
                print('Nouvelle moyenne du price_buy',_price_buy_mean)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_buy)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_OPEN.append(1) 
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0) 

                '''elif harmonics[h]==-1:
                print(i)
                _position = 0
                _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                _pnl,_flag = convert(_pnl,_rate,_flag)
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                CASH.append(_cash)
                _winner += _open_buy

                TRADE_DURATION.append(i - _index_entry)

                print(_open_buy,'positions TP closed on opposite signal at',price.Close_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy)
                PRICE_BUY = [] 
                _open_buy = 0'''


            else:

                if (float(price.Close_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) >= _target :#slowd[i]>80:
                #if (rsi[i]>rsi_high):
                    print(i)
                    _position = 0
                    _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _winner += _open_buy

                    TRADE_DURATION.append(i - _index_entry)
                    print(i)
                    print(_open_buy,'positions TP closed at',price.Close_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_buy)
                    PRICE_BUY = [] 
                    _open_buy = 0

                elif (float(price.Close_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) <= -_sl :
                    print(i)
                    _position = 0
                    _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _looser += _open_buy

                    TRADE_DURATION.append(i - _index_entry)
                    print(i)
                    print(_open_buy,'positions SL closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_buy) 
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    PRICE_BUY = []
                    _open_buy = 0
                else:

                    _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _equity = _cash + _pnl
                    EQUITY.append(_equity)
                    CASH.append(_cash)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(0)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(0) ############### 
                    PNL_LAT.append(_pnl)
                    PNL_REAL.append(0)
                    TOTAL_PNL_LAT.append(_pnl)
                    TOTAL_PNL_REAL.append(0) 


        elif _position == -1:

            
            if ( price.Close[i] - price_sell)/price_sell>=_filtre_multipoz and _minipoz == 0 and price.iloc[i]['Close']<MM21[i] and MM21[i]<MM34[i]<MM54[i]\
            and price2.Close[j]<MM542[j]:
                '''if harmonics[h]==-1 and ( price.Close[i] - price_sell)/price_sell>=_filtre_multipoz and _minipoz == 0 and price.iloc[i]['Close']<MM21[i] and X>MM54[current_idx[0]] and MM21[current_idx[0]]>MM54[current_idx[0]] and MM21[i]<MM34[i]<MM54[i]\
            and price2.Close[j]<MM542[j]:'''
            #and price2.Close[j] < MM342[j] : #and price[i]<lowerband[i]:#and rsi[i]<=rsi_low:
                print(i)
                _minipoz = -1
                _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                _open_sell += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _position = -1
                _index_entry = i
                _nbtransactions += 1
                price_sell = price.Close_Bid.iloc[i]
                PRICE_SELL.append(price_sell)
                _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')
                print('Nouvelle moyenne du price_sell',_price_sell_mean)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_sell)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_OPEN.append(1) 
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0) 

                '''elif harmonics[h]==1:
                print(i)
                _position = 0
                _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                _pnl,_flag = convert(_pnl,_rate,_flag)
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                CASH.append(_cash)
                _winner += _open_sell
                TRADE_DURATION.append(i - _index_entry)
                print(i)
                print(_open_sell,'positions TP closed on opposite signal at',price.Close_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = [] 
                _open_sell = 0'''

            else:

                if (float(price.Close_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) <= -_target :
                #if(rsi[i]<rsi_low):
                    print(i)
                    _position = 0
                    _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _winner += _open_sell
                    TRADE_DURATION.append(i - _index_entry)
                    print(i)
                    print(_open_sell,'positions TP closed at',price.Close_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(-1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_sell)
                    PRICE_SELL = [] 
                    _open_sell = 0

                elif (float(price.Close_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) >= _sl :
                    print(i)
                    _position = 0
                    _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _looser += _open_sell
                    TRADE_DURATION.append(i - _index_entry)
                    print(i)
                    print(_open_sell,'position SL closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(-1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_sell)
                    PRICE_SELL = []
                    _open_sell = 0

                else:

                    _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _equity = _cash + _pnl

                    EQUITY.append(_equity)
                    CASH.append(_cash)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(0)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(0)
                    PNL_LAT.append(_pnl)
                    PNL_REAL.append(0)
                    TOTAL_PNL_LAT.append(_pnl)

                    TOTAL_PNL_REAL.append(0)
                        
                    
                           
                           
        elif i == (len(price)-1) and _position != 0 :


            if _position == -1:
                _position = 0
                _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                _pnl,_flag = convert(_pnl,_rate,_flag)
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                CASH.append(_cash)
                _looser += _open_sell
                TRADE_DURATION.append(i - _index_entry)
                print(i)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_sell,'position closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0

            if _position == 1:

                _position = 0
                _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                _pnl,_flag = convert(_pnl,_rate,_flag)
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                CASH.append(_cash)
                _looser += _open_buy

                TRADE_DURATION.append(i - _index_entry)
                print(i)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_buy,'positions closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0

    try:
        _average_duration = round(statistics.mean(TRADE_DURATION),2)
    
    except:
        print("(pas de duration)") 
        _average_duration = 'NA'               
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.GREEN,round(_total,2),'!. En ',_nbtransactions,' transactions.',col.Style.RESET_ALL ) 
    print(col.Fore.GREEN,"\nNbre de winners :",_winner,col.Style.RESET_ALL)
    print(col.Fore.RED,"\nNbre de loosers :",_looser,col.Style.RESET_ALL)
    print("\nAverage trade duration ",_average_duration)
    backtest_graph['Equity'] = EQUITY
    
    df_resultats[x] = [_period,_period2,(round(_equity,2)),(_winner),(_looser),(_average_duration),(_total)]

    print('Total des gains faramineux :',round(EQUITY[-1] - _cash_ini,2))
    print('Pnl latent restant',_pnl,'Open Buy',_open_buy,'Open Sell',_open_sell)
    DER_POZ.append(_pnl)
    

try:
    os.mkdir('Period_'+_period+'_Period2_'+_period2+'_target_'+str(_target)+'_sl_'+str(_sl)+'_rsiHigh'+str(rsi_high)+'_rsiLow_'+str(rsi_low)+'/')
except:
    pass

df_resultats.T.to_csv('Period_'+_period+'_Period2_'+_period2+'_target_'+str(_target)+'_sl_'+str(_sl)+'_rsiHigh'+str(rsi_high)+'_rsiLow_'+str(rsi_low)+'/pnl_'+str(_total)+'.csv')
    
engine.say("The job is done. Check, we are billionaire")
engine.runAndWait()    


 Le rate du ticker EUR/USD est à  1 
 Scraping des données OHLC pour le ticker  EUR/USD 
 Scraping des données OHLC pour le ticker  EUR/USD 
 Scraping des données OHLC pour le ticker  EUR/USD 
Position 1 bought at 1.08045 (verification liste 1.08045 )
Nouvelle moyenne du price_buy 1.08045
1009
1009
1 positions TP closed at 1.08571  pnl 263.0000000000021 
nombre de candles en position : 8
Equity : 200263.0
Position 1 bought at 1.08733 (verification liste 1.08733 )
Nouvelle moyenne du price_buy 1.08733
2673
2673
1 positions TP closed at 1.09399  pnl 333.0000000000055 
nombre de candles en position : 2
Equity : 200596.0
Position 1 sold at 1.09075 (verification liste 1.09075 )
Nouvelle moyenne du price_sell 1.09075
2878
2878
1 position SL closed at 1.09365  pnl -144.99999999999514 
nombre de candles en position : 5
Equity : 200451.0
Position 1 sold at 1.08597 (verification liste 1.08597 )
Nouvelle moyenne du price_sell 1.08597
3004
3004
1 position SL closed at 1.08796  pnl -99.499999999996

In [12]:
data


,Open_Bid,Open_Ask,Open,High_Bid,High_Ask,High,Low_Bid,Low_Ask,Low,Close_Bid,Close_Ask,Close
date,,,,,,,,,,,,
2020-01-22 03:00:00,1.68057,1.68091,1.680740,1.68142,1.68179,1.681605,1.68051,1.68087,1.680690,1.68078,1.68113,1.680955
2020-01-22 03:30:00,1.68078,1.68113,1.680955,1.68186,1.68223,1.682045,1.68073,1.68112,1.680925,1.68129,1.68163,1.681460
2020-01-22 04:00:00,1.68129,1.68163,1.681460,1.68148,1.68189,1.681685,1.68089,1.68130,1.681095,1.68120,1.68159,1.681395
2020-01-22 04:30:00,1.68120,1.68159,1.681395,1.68217,1.68250,1.682335,1.68119,1.68159,1.681390,1.68142,1.68180,1.681610
2020-01-22 05:00:00,1.68142,1.68180,1.681610,1.68164,1.68198,1.681810,1.68077,1.68111,1.680940,1.68077,1.68111,1.680940
...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-05 20:00:00,1.74823,1.74863,1.748430,1.74890,1.74925,1.749075,1.74701,1.74744,1.747225,1.74703,1.74747,1.747250
2020-11-05 20:30:00,1.74702,1.74747,1.747245,1.74768,1.74804,1.747860,1.74605,1.74642,1.746235,1.74626,1.74673,1.746495
2020-11-05 21:00:00,1.74625,1.74672,1.746485,1.74736,1.74780,1.747580,1.74574,1.74612,1.745930,1.74699,1.74726,1.747125


In [13]:
print(col.Fore.BLUE,'Total des gains faramineux :',round(EQUITY[-1] - _cash_ini,2),col.Style.RESET_ALL)
print(col.Fore.YELLOW,'Total du cash titanesque :',round(EQUITY[-1],2),col.Style.RESET_ALL)

df_resultats.T

 Total des gains faramineux : 6080.6 
 Total du cash titanesque : 206080.6 


,TimeFrame 1,TimeFrame2,Equity,Nbre Winners,Nbre Loosers,Average lenght of trade,Cumul pnl
EUR/USD,m30,H1,200742,5,5,15.6,742.5
GBP/USD,m30,H1,202196,7,2,25.88,1453
EUR/GBP,m30,H1,202164,6,11,15.62,-31.55
EUR/JPY,m30,H1,201769,4,9,18.83,-395.31
GBP/JPY,m30,H1,202197,8,9,11.27,428.66
EUR/AUD,m30,H1,202971,8,8,23.27,773.99
EUR/CAD,m30,H1,203360,10,10,17.67,388.47
AUD/JPY,m30,H1,203697,5,5,13,337.34
CAD/JPY,m30,H1,203570,7,12,21.69,-127.58
NZD/JPY,m30,H1,203287,7,17,9.84,-282.25


In [14]:
df_historical = pd.DataFrame()
df_historical = pd.DataFrame(index=DATE)
df_historical['Contract'] = CONTRACT
df_historical['Open_Poz'] = OPEN_POZ
df_historical['Close_Pos'] = CLOSE_POZ
df_historical['Rate_Open_Poz'] = RATE_OPEN_POZ 
df_historical['Rate_Close_Poze'] = RATE_CLOSE_POZ
df_historical['Pnl_Lat'] = TOTAL_PNL_LAT
df_historical['Pnl_Real'] = TOTAL_PNL_REAL
df_historical.sort_index()
df_historical

,Contract,Open_Poz,Close_Pos,Rate_Open_Poz,Rate_Close_Poze,Pnl_Lat,Pnl_Real
2020-02-21 11:00:00,EUR/USD,1,0,1.08045,0.00000,0.00,0.00
2020-02-21 11:30:00,EUR/USD,0,0,0.00000,0.00000,5.50,0.00
2020-02-21 12:00:00,EUR/USD,0,0,0.00000,0.00000,-13.50,0.00
2020-02-21 12:30:00,EUR/USD,0,0,0.00000,0.00000,-42.00,0.00
2020-02-21 13:00:00,EUR/USD,0,0,0.00000,0.00000,-20.50,0.00
...,...,...,...,...,...,...,...
2020-11-03 07:30:00,EUR/NZD,0,0,0.00000,0.00000,101.18,0.00
2020-11-03 08:00:00,EUR/NZD,0,0,0.00000,0.00000,136.04,0.00
2020-11-03 08:30:00,EUR/NZD,0,0,0.00000,0.00000,178.00,0.00
2020-11-03 09:00:00,EUR/NZD,0,0,0.00000,0.00000,135.02,0.00


In [16]:
print('Cash généré :',str(round(df_historical.Pnl_Real.sum(),2))+'$')
print('Pnl latent minimal :',str(df_historical.Pnl_Real.min())+'$')
print('Pnl latent maximal :',str(df_historical.Pnl_Real.max())+'$')
print('Pnl latent moyen :',str(round(df_historical.Pnl_Real.mean(),2))+'$')
print('Max des durées moyennes',df_resultats.T['Average lenght of trade'][df_resultats.T['Average lenght of trade'] != 'NA'].max())
print('Min des durées moyennes',df_resultats.T['Average lenght of trade'][df_resultats.T['Average lenght of trade'] != 'NA'].min())
print('Moyenne des durées moyennes',round(df_resultats.T['Average lenght of trade'][df_resultats.T['Average lenght of trade'] != 'NA'].mean(),2))
print('Nombre total de winners',df_resultats.T['Nbre Winners'].sum())
print('Nombre total de loosers',df_resultats.T['Nbre Loosers'].sum())
print('Total des gains cumulés',round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum(),2))
print('Total des pertes cumulées',round(df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2))
print(col.Fore.BLUE,'PROFIT FACTOR : ',\
    abs(round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum()/df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2)),col.Style.RESET_ALL)
print(col.Fore.CYAN,'Ratio du nombre de winners :',\
    round((df_resultats.T['Nbre Winners'].sum()*100)/(df_resultats.T['Nbre Loosers'].sum()+df_resultats.T['Nbre Winners'].sum()),2),'%',col.Style.RESET_ALL)
print('Nombre total de trades ouverts',sum(TOTAL_OPEN))
print('Nombre total de trades fermés',sum(TOTAL_CLOSE))
print("Nombre de positions en l'air",(sum(TOTAL_OPEN) - sum(TOTAL_CLOSE)))


Cash généré : 6080.6$
Pnl latent minimal : -494.39$
Pnl latent maximal : 902.52$
Pnl latent moyen : 1.52$
Max des durées moyennes 25.88
Min des durées moyennes 9.84
Moyenne des durées moyennes 17.6
Nombre total de winners 103
Nombre total de loosers 136
Total des gains cumulés 21589.35
Total des pertes cumulées -15508.75
 PROFIT FACTOR :  1.39 
 Ratio du nombre de winners : 43.1 % 
Nombre total de trades ouverts 239
Nombre total de trades fermés 239
Nombre de positions en l'air 0


In [ ]:
#########################################@

In [ ]:
###########################################@

In [ ]:
### NETTOYAGE

In [ ]:
########################################

In [42]:
########################################
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import pandas as pd
from numpy import loadtxt
from functools import reduce
import talib
import time
from scipy.signal import argrelextrema
from ipywidgets import *

from datetime import datetime

DATE = []
CONTRACT = []
OPEN_POZ = []
CLOSE_POZ = []
RATE_OPEN_POZ = []
RATE_CLOSE_POZ = []
PNL_LAT = []
PNL_REAL = []
TOTAL_OPEN = []
TOTAL_CLOSE = []
PRICE_BUY = []
PRICE_SELL = []
DER_POZ = []
TOTAL_PNL_LAT = []
TOTAL_PNL_REAL = []
_cash = _cash_ini

In [43]:
def peak_detect(price,order=10):
    max_idx = list(argrelextrema(price,np.greater,order=order)[0])
    min_idx = list(argrelextrema(price,np.less,order=order)[0])
    idx = max_idx + min_idx + [len(price)-1]
    idx.sort()
    _current_idx = idx[-3:]
    
    _start = min(_current_idx)
    _end = max(_current_idx)
    _current_pat = price[_current_idx]
    return _current_idx,_current_pat,_start,_end 
    
def is_BAAM(moves,err_allowed):  
    XA=moves[0]
    AB=moves[1]
    
    
    X=prices[0]
    A=prices[1]
    B=prices[2]
    
    AB_range = np.array([0.15 - err_allowed,0.15+ err_allowed])*abs(XA)
    AB_range2 = np.array([0.38 - err_allowed,0.38+ err_allowed])*abs(XA)
    
    
    
    if abs(A-X)/X>0.001:
        
        if XA>0 and AB<0 :

            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]) :

                BAAM = 1
                return (BAAM)
            else:
                return (np.isnan)

        elif XA<0 and AB>0 :


            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]):

                BAAM = -1
                return (BAAM)
            else:
                return (np.isnan)
        else:
             return (np.isnan)    

In [44]:
%%time
df_resultats = pd.DataFrame(index=['TimeFrame 1','TimeFrame2','Equity','Nbre Winners','Nbre Loosers','Average lenght of trade','Cumul pnl'])

#TICKERS = [ 'EUR/USD','GBP/USD','EUR/JPY','GBP/JPY','EUR/AUD','GBP/AUD','EUR/NZD']

TICKERS = [ 'EUR/USD']

engine.say("קדימה")
engine.runAndWait()

err_allowed = 2/100
plt.ion()
position = 0
equity = 100000
for x in TICKERS:

    if x[4:] == 'USD':
        _rate = 1

    elif x[:3] == 'USD':
        con.subscribe_market_data(x)
        _rate = con.get_last_price(x).Bid
        con.unsubscribe_market_data(x)
    elif x[:3] != 'USD' and x[4:] != 'USD':
        try:
            con.subscribe_market_data(x[4:]+'/USD')
            _rate = con.get_last_price(x[4:]+'/USD').Bid
            con.unsubscribe_market_data(x[4:]+'/USD')
            _flag = 1
        except:
            con.subscribe_market_data('USD/'+x[4:])
            _rate = con.get_last_price('USD/'+x[4:]).Bid
            con.unsubscribe_market_data('USD/'+x[4:])
            _flag = -1
    else:
        prix_usd = 'NA'

    print(col.Fore.MAGENTA,'Le rate du ticker',x,'est à ',_rate,col.Style.RESET_ALL)
    label = ''
    data = scrap_hist(x,_number,_period)
    data2 = scrap_hist(x,_number,_period2)
    price = data.copy()
    price2 = data2.copy()
    _position = 0
    _equity = 0
    _nbtransactions = 0
    backtest_graph = pd.DataFrame()
    EQUITY = [_cash]
    CASH = [_cash]
    _winner = 0
    _looser = 0
    _index_entry = 0
    TRADE_DURATION = []
    _average_duration = 0
    _size = 50000

    PRICE_BUY = []
    PRICE_SELL = []
   
    _total = 0

    _target = 0.006
    _sl = 0.003
    _spread_filter = 0.00050
    _filtre_multipoz = 0.01 # En %
    _filtre_multipoz = _filtre_multipoz/100
    
    _open_buy = 0
    _open_sell = 0
    _minipoz = 0
    
    MM21 = talib.EMA(price.Close, timeperiod=21)
    MM34 = talib.EMA(price.Close, timeperiod=34)
    MM54 = talib.EMA(price.Close, timeperiod=54)
    rsi = talib.RSI(price.Close, timeperiod=14)
    rsi_high = 70
    rsi_low = 30
    
    MM542 = talib.EMA(price2.Close, timeperiod=54)
    
    for i in range(150,len(price)-1):
        
        j = len(price2[price2.index <= price.index[i]])-1
        
        current_idx,current_pat,start,end = peak_detect(price.Close.values[:i+1])
        current_rsi = [rsi.iloc[current_idx[0]],rsi.iloc[current_idx[1]],rsi.iloc[current_idx[2]]]
        current_rsi_max = max(current_rsi)
        current_rsi_min = min(current_rsi)
        delta_price = current_pat[2] - current_pat[0]
        delta_rsi = current_rsi[2] - current_rsi[0]
      
        XA = current_pat[1] - current_pat[0]     
        AB = current_pat[2] - current_pat[1]      
        
        X = current_pat[0]
        A = current_pat[1]
        B = current_pat[2]

        prices =[X,A,B]

        moves=[XA,AB]
        
        BAAM = is_BAAM(moves,err_allowed)

        harmonics = np.array([BAAM])

        if _position == 0 :
            if (np.any(harmonics==1) and np.all(harmonics!=-1 )) or  (np.any(harmonics==-1) and np.all(harmonics!=1 )):

                for h in range(0, len(harmonics)):

                    if harmonics[h]==-1 and price.iloc[i]['Close']<MM21[i] and X>MM54[current_idx[0]] and MM21[current_idx[0]]>MM54[current_idx[0]] and MM21[i]<MM34[i]<MM54[i]\
                            and price2.Close[j]<MM542[j]:
                            _pnl = 0
                            _open_sell += 1
                            _equity = _cash + _pnl
                            EQUITY.append(_equity)
                            CASH.append(_cash)
                            _index_entry = i
                            _position = -1
                            _nbtransactions += 1
                            price_sell = price.Close_Bid.iloc[i]
                            PRICE_SELL.append(price_sell)
                            _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
                            print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')
                            print('Nouvelle moyenne du price_sell',_price_sell_mean)

                            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                            CONTRACT.append(x)
                            OPEN_POZ.append(-1)
                            CLOSE_POZ.append(0)
                            RATE_OPEN_POZ.append(price_sell)
                            RATE_CLOSE_POZ.append(0) 
                            PNL_LAT.append(_pnl)
                            PNL_REAL.append(0)
                            TOTAL_PNL_LAT.append(_pnl)
                            TOTAL_PNL_REAL.append(0)

                            TOTAL_OPEN.append(1)
                    
                    if harmonics[h]==1 and price.iloc[i]['Close']>MM21[i] and X<MM54[current_idx[0]] and MM21[current_idx[0]]<MM54[current_idx[0]] and MM21[i]>MM34[i]>MM54[i]\
                        and price2.Close[j]>MM542[j]:
                   
                            _pnl = 0
                            _open_buy += 1
                            _equity = _cash + _pnl
                            EQUITY.append(_equity)
                            CASH.append(_cash)
                            _position = 1
                            _index_entry = i
                            _nbtransactions += 1
                            price_buy = price.Close_Ask.iloc[i]
                            PRICE_BUY.append(price_buy)
                            _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
                            print('Position 1 bought at', price_buy,'(verification liste',PRICE_BUY[-1],')')
                            print('Nouvelle moyenne du price_buy',_price_buy_mean)

                            DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                            CONTRACT.append(x)
                            OPEN_POZ.append(1)
                            CLOSE_POZ.append(0)
                            RATE_OPEN_POZ.append(price_buy)
                            RATE_CLOSE_POZ.append(0)
                            PNL_LAT.append(_pnl)
                            PNL_REAL.append(0)
                            TOTAL_OPEN.append(1) 
                            TOTAL_PNL_LAT.append(_pnl)
                            TOTAL_PNL_REAL.append(0) 

                    else :
                        _pnl = 0
                        _equity = _cash + _pnl
                        EQUITY.append(_equity)
                        CASH.append(_cash)
                
        elif _position == 1:

            
            if (price_buy - price.Close[i])/price_buy>=_filtre_multipoz and  MM21[i]>MM34[i]>MM54[i]\
            and price2.Close[j]>MM542[j] :
             
                _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                _open_buy += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _position = 1
                _index_entry = i
                _nbtransactions += 1
                price_buy = price.Close_Ask.iloc[i]
                PRICE_BUY.append(price_buy)
                _price_buy_mean = round(sum(PRICE_BUY)/len(PRICE_BUY),5)
                print('Position',_open_buy,'bought at ', price_buy,'(verification liste',PRICE_BUY[-1],')')
                print('Nouvelle moyenne du price_buy',_price_buy_mean)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_buy)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_OPEN.append(1) 
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0) 


            else:

                if (float(price.Close_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) >= _target :#slowd[i]>80:
               
                    _position = 0
                    _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _winner += _open_buy

                    TRADE_DURATION.append(i - _index_entry)
                    print(_open_buy,'positions TP closed at',price.Close_Bid.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_buy)
                    PRICE_BUY = [] 
                    _open_buy = 0

                elif ((float(price.Close_Bid.iloc[i]) - float(_price_buy_mean))/float(_price_buy_mean) <= -_sl) or \
                    (PNL_LAT[-1] < -180):
                    _position = 0
                    _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _looser += _open_buy

                    TRADE_DURATION.append(i - _index_entry)
                    print(_open_buy,'positions SL closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_buy) 
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    PRICE_BUY = []
                    _open_buy = 0
                else:

                    _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _equity = _cash + _pnl
                    EQUITY.append(_equity)
                    CASH.append(_cash)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(0)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(0) ############### 
                    PNL_LAT.append(_pnl)
                    PNL_REAL.append(0)
                    TOTAL_PNL_LAT.append(_pnl)
                    TOTAL_PNL_REAL.append(0) 


        elif _position == -1:

            
            if ( price.Close[i] - price_sell)/price_sell>=_filtre_multipoz and price.iloc[i]['Close']<MM21[i] and MM21[i]<MM34[i]<MM54[i]\
            and price2.Close[j]<MM542[j]:
                
                _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                _open_sell += 1
                _equity = _cash + _pnl
                EQUITY.append(_equity)
                CASH.append(_cash)
                _position = -1
                _index_entry = i
                _nbtransactions += 1
                price_sell = price.Close_Bid.iloc[i]
                PRICE_SELL.append(price_sell)
                _price_sell_mean = round(sum(PRICE_SELL)/len(PRICE_SELL),5)
                print('Position 1 sold at', price_sell,'(verification liste',PRICE_SELL[-1],')')
                print('Nouvelle moyenne du price_sell',_price_sell_mean)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(1)
                CLOSE_POZ.append(0)
                RATE_OPEN_POZ.append(price_sell)
                RATE_CLOSE_POZ.append(0)
                PNL_LAT.append(_pnl)
                PNL_REAL.append(0)
                TOTAL_OPEN.append(1) 
                TOTAL_PNL_LAT.append(_pnl)
                TOTAL_PNL_REAL.append(0) 


            else:

                if (float(price.Close_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) <= -_target :
                    _position = 0
                    _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _winner += _open_sell
                    TRADE_DURATION.append(i - _index_entry)
                    print(_open_sell,'positions TP closed at',price.Close_Ask.iloc[i],col.Fore.GREEN,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(-1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_sell)
                    PRICE_SELL = [] 
                    _open_sell = 0

                elif ((float(price.Close_Ask.iloc[i]) - float(_price_sell_mean))/float(_price_sell_mean) >= _sl) or \
                    (PNL_LAT[-1] < -180):
                    _position = 0
                    _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _total += _pnl
                    _cash += _pnl
                    _equity = _cash
                    EQUITY.append(_equity)
                    CASH.append(_cash)
                    _looser += _open_sell
                    TRADE_DURATION.append(i - _index_entry)
                    print(_open_sell,'position SL closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                    print('nombre de candles en position :',i - _index_entry)
                    print('Equity :', _equity)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(-1)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                    PNL_LAT.append(0)
                    PNL_REAL.append(_pnl)
                    TOTAL_PNL_LAT.append(0)
                    TOTAL_PNL_REAL.append(_pnl)
                    TOTAL_CLOSE.append(_open_sell)
                    PRICE_SELL = []
                    _open_sell = 0

                else:

                    _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                    _pnl,_flag = convert(_pnl,_rate,_flag)
                    _equity = _cash + _pnl

                    EQUITY.append(_equity)
                    CASH.append(_cash)

                    DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                    CONTRACT.append(x)
                    OPEN_POZ.append(0)
                    CLOSE_POZ.append(0)
                    RATE_OPEN_POZ.append(0)
                    RATE_CLOSE_POZ.append(0)
                    PNL_LAT.append(_pnl)
                    PNL_REAL.append(0)
                    TOTAL_PNL_LAT.append(_pnl)

                    TOTAL_PNL_REAL.append(0)
              
        elif i == (len(price)-1) and _position != 0 :

            if _position == -1:
                _position = 0
                _pnl = - (price.Close_Ask.iloc[i] - _price_sell_mean) * _size * _open_sell
                _pnl,_flag = convert(_pnl,_rate,_flag)
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                CASH.append(_cash)
                _looser += _open_sell
                TRADE_DURATION.append(i - _index_entry)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_sell,'position closed at',price.Close_Ask.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(-1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.Close_Ask.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_sell)
                PRICE_SELL = []
                _open_sell = 0

            if _position == 1:

                _position = 0
                _pnl = (price.Close_Bid.iloc[i] - _price_buy_mean) * _size * _open_buy
                _pnl,_flag = convert(_pnl,_rate,_flag)
                _total += _pnl
                _cash += _pnl
                _equity = _cash
                EQUITY.append(_equity)
                CASH.append(_cash)
                _looser += _open_buy

                TRADE_DURATION.append(i - _index_entry)
                print(col.Fore.CYAN,"Cloture des positions en l'air",col.Style.RESET_ALL)
                print(_open_buy,'positions closed at',price.Close_Bid.iloc[i],col.Fore.RED,'pnl', _pnl,col.Style.RESET_ALL)
                print('nombre de candles en position :',i - _index_entry)
                print('Equity :', _equity)

                DATE.append(price.index[i].strftime(format='%Y-%m-%d %H:%M:%S'))
                CONTRACT.append(x)
                OPEN_POZ.append(0)
                CLOSE_POZ.append(1)
                RATE_OPEN_POZ.append(0)
                RATE_CLOSE_POZ.append(price.Close_Bid.iloc[i])
                PNL_LAT.append(0)
                PNL_REAL.append(_pnl)
                TOTAL_CLOSE.append(_open_buy) 
                TOTAL_PNL_LAT.append(0)
                TOTAL_PNL_REAL.append(_pnl)
                PRICE_BUY = []
                _open_buy = 0

    try:
        _average_duration = round(statistics.mean(TRADE_DURATION),2)
    
    except:
        print("(pas de duration)") 
        _average_duration = 'NA' 
    print(col.Fore.BLUE,'Pour le ticker',col.Fore.YELLOW,x,col.Style.RESET_ALL)              
    print(col.Fore.MAGENTA,"\nLes gains faramineux s'élèvent à : $",col.Fore.GREEN,round(_total,2),'!. En ',_nbtransactions,' transactions.',col.Style.RESET_ALL ) 
    print(col.Fore.GREEN,"\nNbre de winners :",_winner,col.Style.RESET_ALL)
    print(col.Fore.RED,"\nNbre de loosers :",_looser,col.Style.RESET_ALL)
    print("\nAverage trade duration ",_average_duration)
    backtest_graph['Equity'] = EQUITY
    
    df_resultats[x] = [_period,_period2,(round(_equity,2)),(_winner),(_looser),(_average_duration),(_total)]

    print('Total des gains faramineux :',round(EQUITY[-1] - _cash_ini,2))
    print('Pnl latent restant',_pnl,'Open Buy',_open_buy,'Open Sell',_open_sell)
    DER_POZ.append(_pnl)



 Le rate du ticker EUR/USD est à  1 
 Scraping des données OHLC pour le ticker  EUR/USD 
 Scraping des données OHLC pour le ticker  EUR/USD 
Position 1 bought at 1.10164 (verification liste 1.10164 )
Nouvelle moyenne du price_buy 1.10164
Position 2 bought at  1.10127 (verification liste 1.10127 )
Nouvelle moyenne du price_buy 1.10146
2 positions TP closed at 1.10862  pnl 715.9999999999833 
nombre de candles en position : 60
Equity : 200715.99999999997
Position 1 bought at 1.08045 (verification liste 1.08045 )
Nouvelle moyenne du price_buy 1.08045
Position 2 bought at  1.08015 (verification liste 1.08015 )
Nouvelle moyenne du price_buy 1.0803
2 positions TP closed at 1.08703  pnl 672.9999999999902 
nombre de candles en position : 62
Equity : 201388.99999999997
Position 1 bought at 1.11885 (verification liste 1.11885 )
Nouvelle moyenne du price_buy 1.11885
Position 2 bought at  1.11874 (verification liste 1.11874 )
Nouvelle moyenne du price_buy 1.1188
2 positions TP closed at 1.12718  pn

In [45]:
df_historical = pd.DataFrame()
df_historical = pd.DataFrame(index=DATE)
df_historical['Contract'] = CONTRACT
df_historical['Open_Poz'] = OPEN_POZ
df_historical['Close_Pos'] = CLOSE_POZ
df_historical['Rate_Open_Poz'] = RATE_OPEN_POZ 
df_historical['Rate_Close_Poze'] = RATE_CLOSE_POZ
df_historical['Pnl_Lat'] = TOTAL_PNL_LAT
df_historical['Pnl_Real'] = TOTAL_PNL_REAL
df_historical = df_historical.sort_index()
print('Cash généré :',str(round(df_historical.Pnl_Real.sum(),2))+'$')
print('Pnl latent minimal :',str(df_historical.Pnl_Real.min())+'$')
print('Pnl latent maximal :',str(df_historical.Pnl_Real.max())+'$')
print('Pnl latent moyen :',str(round(df_historical.Pnl_Real.mean(),2))+'$')
print('Max des durées moyennes',df_resultats.T['Average lenght of trade'][df_resultats.T['Average lenght of trade'] != 'NA'].max())
print('Min des durées moyennes',df_resultats.T['Average lenght of trade'][df_resultats.T['Average lenght of trade'] != 'NA'].min())
print('Moyenne des durées moyennes',round(df_resultats.T['Average lenght of trade'][df_resultats.T['Average lenght of trade'] != 'NA'].mean(),2))
print('Nombre total de winners',df_resultats.T['Nbre Winners'].sum())
print('Nombre total de loosers',df_resultats.T['Nbre Loosers'].sum())
print('Total des gains cumulés',round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum(),2))
print('Total des pertes cumulées',round(df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2))
print(col.Fore.BLUE,'PROFIT FACTOR : ',\
    abs(round(df_historical[df_historical.Pnl_Real>0].Pnl_Real.sum()/df_historical[df_historical.Pnl_Real<0].Pnl_Real.sum(),2)),col.Style.RESET_ALL)
print(col.Fore.CYAN,'Ratio du nombre de winners :',\
    round((df_resultats.T['Nbre Winners'].sum()*100)/(df_resultats.T['Nbre Loosers'].sum()+df_resultats.T['Nbre Winners'].sum()),2),'%',col.Style.RESET_ALL)
print('Nombre total de trades ouverts',sum(TOTAL_OPEN))
print('Nombre total de trades fermés',sum(TOTAL_CLOSE))
print("Nombre de positions en l'air",(sum(TOTAL_OPEN) - sum(TOTAL_CLOSE)))

Cash généré : 14378.0$
Pnl latent minimal : -568.4999999999941$
Pnl latent maximal : 2880.000000000038$
Pnl latent moyen : 7.3$
Max des durées moyennes 34.42
Min des durées moyennes 34.42
Moyenne des durées moyennes 34.42
Nombre total de winners 55
Nombre total de loosers 73
Total des gains cumulés 20546.5
Total des pertes cumulées -6168.5
 PROFIT FACTOR :  3.33 
 Ratio du nombre de winners : 42.97 % 
Nombre total de trades ouverts 128
Nombre total de trades fermés 128
Nombre de positions en l'air 0


In [41]:
%%time
toto = data.copy().reset_index()
order = 10
toto['ExtremClose'] = 0
toto['ExtremClose'].loc[argrelextrema(toto.Close.values,np.greater,order=order)[0]] = 1
toto['ExtremClose'].loc[argrelextrema(toto.Close.values,np.less,order=order)[0]] = -1

CPU times: user 9.46 ms, sys: 3.11 ms, total: 12.6 ms
Wall time: 11.2 ms


,date,Open_Bid,Open_Ask,Open,High_Bid,High_Ask,High,Low_Bid,Low_Ask,Low,Close_Bid,Close_Ask,Close,ExtremClose
0,2020-01-22 03:00:00,1.68057,1.68091,1.680740,1.68142,1.68179,1.681605,1.68051,1.68087,1.680690,1.68078,1.68113,1.680955,0
1,2020-01-22 03:30:00,1.68078,1.68113,1.680955,1.68186,1.68223,1.682045,1.68073,1.68112,1.680925,1.68129,1.68163,1.681460,0
2,2020-01-22 04:00:00,1.68129,1.68163,1.681460,1.68148,1.68189,1.681685,1.68089,1.68130,1.681095,1.68120,1.68159,1.681395,0
3,2020-01-22 04:30:00,1.68120,1.68159,1.681395,1.68217,1.68250,1.682335,1.68119,1.68159,1.681390,1.68142,1.68180,1.681610,0
4,2020-01-22 05:00:00,1.68142,1.68180,1.681610,1.68164,1.68198,1.681810,1.68077,1.68111,1.680940,1.68077,1.68111,1.680940,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2020-11-05 20:00:00,1.74823,1.74863,1.748430,1.74890,1.74925,1.749075,1.74701,1.74744,1.747225,1.74703,1.74747,1.747250,0
9996,2020-11-05 20:30:00,1.74702,1.74747,1.747245,1.74768,1.74804,1.747860,1.74605,1.74642,1.746235,1.74626,1.74673,1.746495,0
9997,2020-11-05 21:00:00,1.74625,1.74672,1.746485,1.74736,1.74780,1.747580,1.74574,1.74612,1.745930,1.74699,1.74726,1.747125,0
9998,2020-11-05 21:30:00,1.74698,1.74726,1.747120,1.74704,1.75211,1.749575,1.74537,1.74652,1.745945,1.74627,1.75211,1.749190,1


In [22]:
PNL_REAL[-5:]

[0, 0, 0, 0, -148.36]

In [64]:
%%time
BAAM = []
rsi = talib.RSI(toto.Close, timeperiod=14)
order = 10

for i in range(len(toto)):
    max_idx = list(argrelextrema(price.Close.values[:i+1],np.greater,order=order)[0])
    min_idx = list(argrelextrema(price.Close.values[:i+1],np.less,order=order)[0])
    idx = max_idx + min_idx + [len(price.Close.values[:i+1])-1]
    idx.sort()
    _current_idx = idx[-3:]

    _start = min(_current_idx)
    _end = max(_current_idx)
    _current_pat = price.Close.values[:i+1][_current_idx]

    current_rsi = [rsi.iloc[current_idx[0]],rsi.iloc[current_idx[1]],rsi.iloc[current_idx[2]]]
    current_rsi_max = max(current_rsi)
    current_rsi_min = min(current_rsi)
    delta_price = current_pat[2] - current_pat[0]
    delta_rsi = current_rsi[2] - current_rsi[0]

    XA = current_pat[1] - current_pat[0]     
    AB = current_pat[2] - current_pat[1]      


    X = current_pat[0]
    A = current_pat[1]
    B = current_pat[2]


    prices =[X,A,B]



    moves=[XA,AB]

    XA=moves[0]
    AB=moves[1]


    X=prices[0]
    A=prices[1]
    B=prices[2]

    AB_range = np.array([0.382 - err_allowed,0.382+ err_allowed])*abs(XA)
    AB_range2 = np.array([0.5 - err_allowed,0.5+ err_allowed])*abs(XA)



    if abs(A-X)/X>0.002:
        
        if XA>0 and AB<0 :

            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]) :

                BAAM.append(1)
                
            else:
                BAAM.append(np.isnan)

        elif XA<0 and AB>0 :


            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]):

                BAAM.append(-1)
                
            else:
                BAAM.append(np.isnan)
        else:
                BAAM.append(np.isnan)       



    harmonics = np.array([BAAM])


CPU times: user 13.2 s, sys: 247 ms, total: 13.4 s
Wall time: 13.4 s


In [65]:
sum(BAAM),-len(BAAM)

(-10000, -10000)

In [ ]:
def peak_detect(price,order=10):
    max_idx = list(argrelextrema(price,np.greater,order=order)[0])
    min_idx = list(argrelextrema(price,np.less,order=order)[0])
    idx = max_idx + min_idx + [len(price)-1]
    idx.sort()
    _current_idx = idx[-3:]
    
    _start = min(_current_idx)
    _end = max(_current_idx)
    _current_pat = price[_current_idx]
    return _current_idx,_current_pat,_start,_end 
    
def is_BAAM(moves,err_allowed):  
    XA=moves[0]
    AB=moves[1]
    
    
    X=prices[0]
    A=prices[1]
    B=prices[2]
    
    AB_range = np.array([0.382 - err_allowed,0.382+ err_allowed])*abs(XA)
    AB_range2 = np.array([0.5 - err_allowed,0.5+ err_allowed])*abs(XA)
    
    
    
    if abs(A-X)/X>0.002:
        
        if XA>0 and AB<0 :

            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]) :

                BAAM = 1
                return (BAAM)
            else:
                return (np.isnan)

        elif XA<0 and AB>0 :


            if (AB_range[0]<abs(AB)<AB_range[1]) or (AB_range2[0]<abs(AB)<AB_range2[1]):

                BAAM = -1
                return (BAAM)
            else:
                return (np.isnan)
        else:
             return (np.isnan)       